In [1]:
import itertools
import pulp

In [2]:
N = 15
R = 3
T = 3

# 定数定義
room_size = N // R  # 各ルームの平均人数
participants = list(range(N))  # 参加者のリスト
rooms = list(range(R))  # ルームのリスト
rounds = list(range(T))  # ラウンドのリスト

problem = pulp.LpProblem("Minimize_Pair_Repetition", pulp.LpMinimize)

# 変数定義
x = {(i, j, k): pulp.LpVariable(f"x_{i}_{j}_{k}", 0, 1, pulp.LpBinary)
        for i in participants for j in rooms for k in rounds}  # x[i, j, k]: 参加者 i がラウンド k のルーム j に割り当てられるか

y = {(i1, i2, j, k): pulp.LpVariable(f"y_{i1}_{i2}_{j}_{k}", 0, 1, pulp.LpBinary)
        for i1, i2 in itertools.combinations(participants, 2) for j in rooms for k in rounds} # y[i1, i2, j, k]: 参加者 i1 と i2 がラウンド k のルーム j に一緒にいるか

z = {(i1, i2): pulp.LpVariable(f"z_{i1}_{i2}", 0, T, pulp.LpInteger)
        for i1, i2 in itertools.combinations(participants, 2)}  # z[i1, i2]: 参加者 i1 と i2 が一緒のルームに割り当てられた総回数

# 目的関数: 全ペアの重複回数の最小化
problem += pulp.lpSum(z[i1, i2] for i1 in participants for i2 in participants if i1 < i2)

# 制約 1: 各参加者は各ラウンドで1つのルームにのみ割り当てられる
for i in participants:
    for k in rounds:
        problem += pulp.lpSum(x[i, j, k] for j in rooms) == 1

# 制約 2: 各ルームの人数は均等またはほぼ均等
for j in rooms:
    for k in rounds:
        problem += pulp.lpSum(x[i, j, k] for i in participants) >= room_size
        problem += pulp.lpSum(x[i, j, k] for i in participants) <= room_size + 1

# 制約 3: ペアの重複回数 z[i1, i2] の計算
for i1, i2 in itertools.combinations(participants, 2):
    for j in rooms:
        for k in rounds:
            problem += y[i1, i2, j, k] <= x[i1, j, k]
            problem += y[i1, i2, j, k] <= x[i2, j, k]
            problem += y[i1, i2, j, k] >= x[i1, j, k] + x[i2, j, k] - 1

    problem += z[i1, i2] == pulp.lpSum(
        y[i1, i2, j, k] for j in rooms for k in rounds
    )

problem.solve()

if problem.status == pulp.LpStatusOptimal:
    assignments = []
    for k in rounds:
        round_assignment = {}
        for j in rooms:
            round_assignment[f"room_{j+1}"] = [
                i for i in participants if pulp.value(x[i, j, k]) == 1
            ]
        assignments.append(round_assignment)

assignments

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/apple/Desktop/Github/room-assignment-api/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_g/th1k5jkx66zf5kvxqw66pb6c0000gn/T/53afbfd20cbd4eec9f0b8f7f33ce00a6-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_g/th1k5jkx66zf5kvxqw66pb6c0000gn/T/53afbfd20cbd4eec9f0b8f7f33ce00a6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3008 COLUMNS
At line 13554 RHS
At line 16558 BOUNDS
At line 17744 ENDATA
Problem MODEL has 3003 rows, 1185 columns and 8070 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 945 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 945 strengthened rows, 0 substitutions
Cgl0004I processed model has 2994 rows, 1185 columns (1185 integer 

KeyboardInterrupt: 

15名, 3ルーム, 3ラウンドでも30分以上たっても解が求まらないため、このままだと実用性は低い。